In [1]:
!pip3 install textblob vaderSentiment transformers
!pip3 install torch torchvision torchaudio
!pip3 install tqdm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [3]:
import pandas as pd
file_path = "/content/drive/My Drive/nyt_articles_merged.csv"

dataframe = pd.read_csv(file_path, delimiter=",")
print(dataframe)
snippets = dataframe["snippet"]

                                                  title  \
0     An Ice Cream Mogul and His Fellow Burlingtonia...   
1     Bill Clinton, Back in Iowa, Refuses to Respond...   
2               Donald Trump Brings Gridlock to Vermont   
3     Ad Wars of 2016 Campaign Erupt in a Changing T...   
4     Ted Cruz’s Iowa Bus Tour: Using ‘Star Wars’ to...   
...                                                 ...   
3483  Trump’s Trade Agenda Could Benefit Friends and...   
3484  Trump Chooses New York Doctor to Be Next U.S. ...   
3485  Trump Picks Scott Turner, Little-Known Officia...   
3486  Sebastian Gorka to Return to White House as Ad...   
3487  Labor Secretary Pick Is Rep. Lori Chavez-DeRem...   

                                                snippet  \
0     In a liberal enclave fiercely loyal to Bernie ...   
1     Bill Clinton returned to the campaign trail in...   
2     Donald J. Trump brought gridlock to Burlington...   
3     Candidates and the outside groups supporting t...

In [4]:
dataframe = dataframe.dropna(subset=['snippet'])  # Drop NaN in 'snippet' column

# Optionally, drop rows where 'snippet' is an empty string
dataframe = dataframe[dataframe['snippet'].str.strip() != '']
print(dataframe.shape)  # To verify the number of rows left


(3460, 6)


In [5]:
from transformers import pipeline
import torch
from tqdm import tqdm

sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

snippets_list = snippets.dropna().tolist()  # Drop NaN and convert to list
snippets_list = [str(snippet) for snippet in snippets_list]  # Ensure strings

results = []
for snippet in tqdm(snippets_list, desc="Analyzing Sentiment"):
    try:
        result = sentiment_pipeline(snippet)
        results.append(result[0])  # Append the first result (single example)
    except Exception as e:
        # Handle any errors (optional)
        results.append({"label": "ERROR", "score": 0.0})

# Add sentiment results to DataFrame
dataframe['sentiment_label'] = [res['label'] for res in results]
dataframe['sentiment_score'] = [res['score'] for res in results]

print(dataframe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Analyzing Sentiment:   0%|          | 0/3460 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Analyzing Sentiment: 100%|██████████| 3460/3460 [10:59<00:00,  5.25it/s]

                                                  title  \
0     An Ice Cream Mogul and His Fellow Burlingtonia...   
1     Bill Clinton, Back in Iowa, Refuses to Respond...   
2               Donald Trump Brings Gridlock to Vermont   
3     Ad Wars of 2016 Campaign Erupt in a Changing T...   
4     Ted Cruz’s Iowa Bus Tour: Using ‘Star Wars’ to...   
...                                                 ...   
3483  Trump’s Trade Agenda Could Benefit Friends and...   
3484  Trump Chooses New York Doctor to Be Next U.S. ...   
3485  Trump Picks Scott Turner, Little-Known Officia...   
3486  Sebastian Gorka to Return to White House as Ad...   
3487  Labor Secretary Pick Is Rep. Lori Chavez-DeRem...   

                                                snippet  \
0     In a liberal enclave fiercely loyal to Bernie ...   
1     Bill Clinton returned to the campaign trail in...   
2     Donald J. Trump brought gridlock to Burlington...   
3     Candidates and the outside groups supporting t...

In [6]:
save_path = "/content/drive/My Drive/nyt_articles_sentiment.csv"
dataframe.to_csv(save_path, index=False)